In [ ]:
import os
import random
import uuid
import numpy as np
from faker import Faker
from datetime import datetime
import time
import threading
import pandas as pd

# Initialize Faker with Spanish locale
fake = Faker('es_ES')

# Define geographical boundaries for Madrid
LAT_MIN, LAT_MAX = 40.312, 40.643
LON_MIN, LON_MAX = -3.888, -3.517

# Constants
max_weight_per_order = 50
max_vol_per_order = 100
status = "in_progress"

# Function to generate a random address in Madrid
def generate_random_address():
    street_name = fake.street_name()
    street_number = fake.building_number()
    lat = random.uniform(LAT_MIN, LAT_MAX)
    lon = random.uniform(LON_MIN, LON_MAX)
    address = f"{street_name}, {street_number}, Madrid, Spain"
    return {
        "address": address,
        "lat": lat,
        "lon": lon
    }

# Function to generate a random order
def generate_random_order(client_id):
    order_id = str(uuid.uuid4())
    volume = np.random.uniform(1, max_vol_per_order)
    weight = np.random.uniform(1, max_weight_per_order)
    n_objects = np.random.randint(1, 10)
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")  # Format the timestamp
    order = {
        "client_id": client_id,
        "location": generate_random_address(),
        "order_id": order_id,
        "order": {
            "n_objects": n_objects,
            "volume": volume,
            "weight": weight
        },
        "status": status,
        "timestamp": timestamp  # Use the formatted timestamp
    }
    print("Generated order:", order)  # Print the generated order
    return order

# Function to simulate appending new orders to a JSON file
def append_order_to_file(order, filepath):
    order_df = pd.DataFrame([order])
    # Append the order to the JSON file
    if not os.path.isfile(filepath):
        order_df.to_json(filepath, orient='records', lines=True, mode='w')  # Write with headers
    else:
        order_df.to_json(filepath, orient='records', lines=True, mode='a')  # Append without headers

# Function to generate orders at random intervals and write them to a JSON file
def order_stream_generator(filepath):
    while True:
        client_id = random.randint(100000000000, 999999999999)
        order = generate_random_order(client_id)
        append_order_to_file(order, filepath)
        
        # Sleep for a random amount of time between 5 and 15 minutes
        sleep_time = random.randint(0.1 * 60, 0.5 * 60)
        print(f"Sleeping for {sleep_time / 60} minutes")
        time.sleep(sleep_time)

# Start the order stream generator in a separate thread
file_path = "/Users/borja/Documents/Somniumrema/projects/de/route_optimizer/data/orders.json"  # Update this with the correct path
thread = threading.Thread(target=order_stream_generator, args=(file_path,))
thread.start()
